In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Capstone

Mounted at /content/drive
/content/drive/MyDrive/Capstone


In [2]:
# Packages
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [28]:
# Functions
def look_n_load(path, describe=False, head=False):
    print('Loading', path)
    df = pd.read_csv(path)
    print('\nShape:', df.shape)
    print('\nColumns:', list(df.columns))
    if describe:
      print('\n',df.describe())
    print('\nMissing proportions:\n', df.isna().sum()/len(df))
    if head:
      print('\nData:\n', df.head())
    print()
    return df

def rescale_temp(x):
    if x > 200:
        return x/10
    else:
        return x

def rescale_heart(x):
    if x > 300:
        return x / 100
    else:
        return x

def rescale_pain(x):
    try:
        x = int(x)
        if x > 100:
            return 10
        elif x > 10:
            return x/10
        elif x < 0:
            return 0
        else:
            return x
    except:
        return None

def reclass_nights(x):
  if x['nights'] > 7:
    return 'Beyond a Week'
  elif x['nights'] == 0: 
    return 'Within Today'
  else:
    return 'Within the Week'

def work_hours(x):
    if x >= 8 and x <= 18:
        return 1 
    else:
        return 0

def race_class(x):
  if 'WHITE' in x or 'PORTU' in x:
    return 'WHITE'
  elif 'BLACK' in x:
    return 'BLACK'
  elif 'LATIN' in x:
    return 'LATINO'
  elif 'ASIAN' in x:
    return 'ASIAN'
  else:
    return 'OTHER'

def cardiac_arrest(x):
  try:
    x = str(x)
    if 'cardiac' in x.lower() or 'arrest' in x.lower():
      return 1
    else:
      return 0
  except:
    return 0

def crit_pain(x):
  x = str(x)
  if 'crit' in x.lower():
    return 1
  else: 
    return 0

def cross_tab_ratio(df, x, y='admitted'):
    rename_these = {
        0: 'home',
        1: 'admitted'
    }
    ctab = pd.crosstab(df[x], df[y]).reset_index().rename(columns=rename_these)
    ctab['total'] = ctab['admitted'] + ctab['home']
    ctab['ratio'] = round(ctab['admitted'] / (ctab['total']),3)
    
    print(ctab.plot.bar(x, 'ratio'))
    return ctab

def get_admission(x):
    if x['disposition'] in ['ADMITTED', 'TRANSFER']:
        return 1
    else:
        return 0
        
def get_expired(x):
    if x['disposition'] == 'EXPIRED':
        return 1
    else:
        return 0

In [14]:
edstays = look_n_load('edstays.csv')


Shape: (447712, 9)

Columns: ['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'gender', 'race', 'arrival_transport', 'disposition']

Missing proportions:
 subject_id           0.000000
hadm_id              0.522213
stay_id              0.000000
intime               0.000000
outtime              0.000000
gender               0.000000
race                 0.000000
arrival_transport    0.000000
disposition          0.000000
dtype: float64

Data:
    subject_id     hadm_id   stay_id           intime          outtime gender  \
0    10000032  22595853.0  33258284   6/5/2180 19:17   6/5/2180 23:30      F   
1    10000032  22841357.0  38112554  26/6/2180 15:54  26/6/2180 21:31      F   
2    10000032  25742920.0  35968195   5/8/2180 20:58    6/8/2180 1:44      F   
3    10000032  29079034.0  32952584  22/7/2180 16:24   23/7/2180 5:54      F   
4    10000032  29079034.0  39399961   23/7/2180 5:54  23/7/2180 14:00      F   

    race arrival_transport disposition  
0  WHITE         AMB

In [8]:
print(edstays.columns, edstays_up.columns)


Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'gender',
       'race', 'arrival_transport', 'disposition'],
      dtype='object') Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime',
       'gender', 'race', 'arrival_transport', 'disposition', 'in_date',
       'out_date', 'race_class', 'hour', 'work_hours', 'subjects_entering',
       'subjects_leaving'],
      dtype='object')


In [19]:
edstays['intime'] = pd.to_datetime(edstays.intime)
edstays['outtime'] = pd.to_datetime(edstays.outtime)
edstays['in_date'] = [x.date() for x in edstays['intime']]
edstays['out_date'] = [x.date() for x in edstays['outtime']]
edstays['nights'] = edstays['out_date'] - edstays['in_date']
edstays['nights'] = [x.days for x in edstays['nights']]
edstays['hour'] = [x.hour for x in edstays['intime']]
edstays['work_hours'] = edstays.hour.apply(work_hours)

edstays['race_class'] = edstays.race.apply(race_class)

edstays['length_of_stay'] = edstays.apply(reclass_nights, axis=1)
edstays['admitted'] = edstays.apply(get_admission, axis=1)
edstays['expired'] = edstays.apply(get_expired, axis=1)

In [27]:
# do history once subsetted

In [46]:
triage = look_n_load('triage.csv')

triage['temperature'] = triage.temperature.apply(rescale_temp)
triage['heartrate'] = triage.heartrate.apply(rescale_heart)
triage['critical'] = triage.pain.apply(crit_pain)
triage['cardiac_arrest'] = triage.chiefcomplaint.apply(cardiac_arrest)
triage['pain'] = triage.pain.apply(rescale_pain)

vitals = look_n_load('vitalsign.csv')

vitals['pain'] = vitals.pain.apply(rescale_pain)
vitals = vitals.drop(columns=['charttime','rhythm']).groupby(['subject_id','stay_id']).last().reset_index()

drop_edstays = ['hadm_id', 'intime','outtime','race','out_date'] # keep in_date for last disposition
edstays_sub = edstays.drop(columns=drop_edstays)

join_cols = ['subject_id','stay_id']

vitals.columns = join_cols + ['last_' + x for x in vitals.columns if x not in join_cols]

all_data = edstays_sub.merge(triage, on=join_cols, how='left').merge(vitals, on=join_cols, how='left')
all_data.info()

Loading triage.csv

Shape: (447712, 11)

Columns: ['subject_id', 'stay_id', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint']

Missing proportions:
 subject_id        0.000000
stay_id           0.000000
temperature       0.054935
heartrate         0.040111
resprate          0.047743
o2sat             0.048355
sbp               0.042941
dbp               0.044823
pain              0.030415
acuity            0.016368
chiefcomplaint    0.000051
dtype: float64

Loading vitalsign.csv

Shape: (1646976, 11)

Columns: ['subject_id', 'stay_id', 'charttime', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'rhythm', 'pain']

Missing proportions:
 subject_id     0.000000
stay_id        0.000000
charttime      0.000000
temperature    0.360779
heartrate      0.044546
resprate       0.057115
o2sat          0.086914
sbp            0.051936
dbp            0.051936
rhythm         0.961874
pain           0.283495
dtype: float64

<class 'pand

In [72]:
# impute with medians
impute_cols = ['temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'last_temperature', 'last_heartrate', 'last_resprate',
       'last_o2sat', 'last_sbp', 'last_dbp', 'last_pain']

all_data['chiefcomplaint'] = all_data.chiefcomplaint.fillna('')

medians = all_data[impute_cols].agg(np.median).reset_index()
medians.columns = ['col_name', 'value']
print(medians)
i = 0
for col in impute_cols:
  print(col)
  value = medians[medians['col_name']==col]['value'][i];i+=1
  print(value)
  all_data[col] = all_data[col].fillna(value)

all_data.info()

            col_name  value
0        temperature   98.0
1          heartrate   84.0
2           resprate   18.0
3              o2sat   99.0
4                sbp  133.0
5                dbp   77.0
6               pain    4.0
7             acuity    3.0
8   last_temperature   98.1
9     last_heartrate   77.0
10     last_resprate   17.0
11        last_o2sat   99.0
12          last_sbp  126.0
13          last_dbp   73.0
14         last_pain    0.0
temperature
98.0
heartrate
84.0
resprate
18.0
o2sat
99.0
sbp
133.0
dbp
77.0
pain
4.0
acuity
3.0
last_temperature
98.1
last_heartrate
77.0
last_resprate
17.0
last_o2sat
99.0
last_sbp
126.0
last_dbp
73.0
last_pain
0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 447712 entries, 0 to 447711
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   subject_id         447712 non-null  int64  
 1   stay_id            447712 non-null  int64  
 2   gender             44771

In [77]:
# split up admission, length, mortality datasets

admitted = all_data[all_data.disposition.isin(['ADMITTED','TRANSFER','HOME'])].drop(columns=['length_of_stay','expired'])
length_of_stay = all_data[all_data.disposition.isin(['ADMITTED','TRANSFER'])].drop(columns=['admitted','expired'])
mortality = all_data[all_data.disposition.isin(['ADMITTED','TRANSFER','EXPIRED'])].drop(columns=['length_of_stay','admitted'])

print(admitted.shape, length_of_stay.shape, mortality.shape)
print()
print('Admitted:\n', admitted.admitted.value_counts())
print()
print('Length:\n', length_of_stay.length_of_stay.value_counts())
print()
print('Expired:\n', mortality.expired.value_counts())

(428284, 29) (173739, 29) (174134, 29)

Admitted:
 0    254545
1    173739
Name: admitted, dtype: int64

Length:
 Within Today       115682
Within the Week     37183
Beyond a Week       20874
Name: length_of_stay, dtype: int64

Expired:
 0    173739
1       395
Name: expired, dtype: int64


In [79]:
admitted.to_csv('admitted.csv', index=False)
length_of_stay.to_csv('length_of_stay.csv', index=False)
mortality.to_csv('mortality.csv', index=False)